In [1]:
import os

import itertools
import pickle
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import math 

import sys
sys.path.append('../')
import utils
from wiki2_utils import WikiText2Dataset
%matplotlib inline

#### параметры

In [2]:
batch_size = 128
sequence_length = 30
grad_clip = 0.1
lr = 4.
best_val_loss = None
log_interval = 100

eval_batch_size = 128

#### загрузчики датасетов (с использованием нового класса)

In [3]:
%%time

dataset = WikiText2Dataset(batch_size=batch_size, eval_batch_size=eval_batch_size, 
                           sequence_length=sequence_length)

train_loader = dataset.get_train_loader()
val_loader = dataset.get_val_loader()
test_loader = dataset.get_test_loader()

Wall time: 3 s


In [5]:
class RNNModel(nn.Module):

    def __init__(self, rnn_type, ntoken, ninp, nhid, nlayers, dropout=0.5):
        super(RNNModel, self).__init__()
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp)
        if rnn_type == 'LSTM':
            self.rnn = nn.LSTM(ninp, nhid, nlayers, dropout=dropout)
        elif rnn_type == 'GRU':
            self.rnn = nn.GRU(ninp, nhid, nlayers, dropout=dropout)
        self.decoder = nn.Linear(nhid, ntoken)

        self.init_weights()

        self.rnn_type = rnn_type
        self.nhid = nhid
        self.nlayers = nlayers

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.fill_(0)
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, x, hidden=None):
        emb = self.drop(self.encoder(x))  # 30x128x128(ninp - размерность единицы на входе)
        output, hidden = self.rnn(emb, hidden)  # 30x128x128(nhid - размерность единицы на выходе (внутр. слоя))
        output = self.drop(output)
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        return decoded.view(output.size(0), output.size(1), decoded.size(1)), hidden

    def init_hidden(self, bsz):
        weight = next(self.parameters()).data
        if self.rnn_type == 'LSTM':
            return (weight.new(self.nlayers, bsz, self.nhid).zero_(),
                    weight.new(self.nlayers, bsz, self.nhid).zero_())
        else:
            return weight.new(self.nlayers, bsz, self.nhid).zero_()

In [7]:
def evaluate(data_loader):
    model.eval()
    total_loss = 0
    ntokens = len(dataset.idx2symbol)
    hidden = model.init_hidden(eval_batch_size)
    for i, (data, targets) in enumerate(data_loader):
        output, hidden = model(data)
        output_flat = output.view(-1, ntokens)
        total_loss += len(data) * criterion(output_flat, targets).item()
    return total_loss / len(data_loader)

In [8]:
def train():
    model.train()
    total_loss = 0
    ntokens = len(dataset.idx2symbol)
    for batch, (data, targets) in enumerate(train_loader):
        model.zero_grad()
        output, hidden = model(data)
        loss = criterion(output.view(-1, ntokens), targets)
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
        for p in model.parameters():
            p.data.add_(-lr, p.grad.data)

        total_loss += loss.item()

        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            print('| epoch {:3d} | {:5d}/{:5d} batches | lr {:02.2f} | loss {:5.2f} | ppl {:8.2f}'.format(
                epoch, batch, len(train_loader) // sequence_length, lr, cur_loss, math.exp(cur_loss)))
            total_loss = 0

In [9]:
ntokens = len(dataset.idx2symbol)
model = RNNModel('LSTM', ntokens, 128, 128, 2, 0.3)
criterion = nn.CrossEntropyLoss()

In [10]:
def generate(n=50, temp=1.):
    model.eval()
    x = torch.rand(1, 1).mul(ntokens).long()
    hidden = None
    out = []
    for i in range(n):
        output, hidden = model(x, hidden)
        s_weights = output.squeeze().data.div(temp).exp()
        s_idx = torch.multinomial(s_weights, 1)[0]
        x.data.fill_(s_idx)
        s = dataset.idx2symbol[s_idx]
        out.append(s)
    return ''.join(out)

In [11]:
with torch.no_grad():
    print('sample:\n', generate(50), '\n')

for epoch in range(1, 6):
    train()
    val_loss = evaluate(val_loader)
    print('-' * 89)
    print('| end of epoch {:3d} | valid loss {:5.2f} | valid ppl {:8.2f}'.format(
        epoch, val_loss, math.exp(val_loss)))
    print('-' * 89)
    if not best_val_loss or val_loss < best_val_loss:
        best_val_loss = val_loss
    else:
        # Anneal the learning rate if no improvement has been seen in the validation dataset.
        lr /= 4.0
    with torch.no_grad():
        print('sample:\n', generate(50), '\n')


sample:
 ヴPล?≤'·μëプ大l7к攻μตả์HìαÎễ戦्?ย動ăuIW>½リ‑Ø⁄ôósァ隊e½dúū空 



..\torch\csrc\utils\python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)


| epoch   1 |   100/ 2807 batches | lr 4.00 | loss  3.59 | ppl    36.36
| epoch   1 |   200/ 2807 batches | lr 4.00 | loss  3.29 | ppl    26.72
| epoch   1 |   300/ 2807 batches | lr 4.00 | loss  3.25 | ppl    25.80
| epoch   1 |   400/ 2807 batches | lr 4.00 | loss  3.22 | ppl    25.12
| epoch   1 |   500/ 2807 batches | lr 4.00 | loss  3.18 | ppl    24.11
| epoch   1 |   600/ 2807 batches | lr 4.00 | loss  3.06 | ppl    21.39
| epoch   1 |   700/ 2807 batches | lr 4.00 | loss  2.98 | ppl    19.70
| epoch   1 |   800/ 2807 batches | lr 4.00 | loss  2.90 | ppl    18.20
| epoch   1 |   900/ 2807 batches | lr 4.00 | loss  2.82 | ppl    16.83
| epoch   1 |  1000/ 2807 batches | lr 4.00 | loss  2.74 | ppl    15.48
| epoch   1 |  1100/ 2807 batches | lr 4.00 | loss  2.63 | ppl    13.90
| epoch   1 |  1200/ 2807 batches | lr 4.00 | loss  2.56 | ppl    12.95
| epoch   1 |  1300/ 2807 batches | lr 4.00 | loss  2.50 | ppl    12.24
| epoch   1 |  1400/ 2807 batches | lr 4.00 | loss  2.45 | ppl  

In [12]:
%%time

t1 = generate(10000, 1.)
t15 = generate(10000, 1.5)
t075 = generate(10000, 0.75)
with open('./generated075.txt', 'w', encoding="utf-8") as outf:
    outf.write(t075)
with open('./generated1.txt', 'w', encoding="utf-8") as outf:
    outf.write(t1)
with open('./generated15.txt', 'w', encoding="utf-8") as outf:
    outf.write(t15)

Wall time: 34.1 s


#### <center> Мой разбор generated()

In [15]:
def generate(n=50, temp=1.):
    model.eval()
    x = torch.rand(1, 1).mul(ntokens).long()
    hidden = None
    out = []
    for i in range(n):
        output, hidden = model(x, hidden)
        s_weights = output.squeeze().data.div(temp).exp()
        '''
        генерируется случайный символ согласно вероятностям, 
        соответствующим каждому из этих символов. Причем на входе s_weights, 
        как я понял, может и не быть вероятностями (то есть не обязательно  
        sum(s_weights) = 1, как в нашем случае). В этом случае просто нормируется на 1.
        '''
        s_idx = torch.multinomial(s_weights, 1)[0]
        x.data.fill_(s_idx)
        s = dataset.idx2symbol[s_idx]
        out.append(s)
    return ''.join(out)

In [127]:
torch.rand(1, 1).mul(ntokens).long()

tensor([[268]])

In [128]:
x = torch.rand(2, 3).mul(ntokens).long()
x

tensor([[ 67,  30, 111],
        [ 47, 207, 143]])

In [129]:
hidden = None
out = []

In [130]:
output, hidden = model(x, hidden)

In [131]:
ntokens

286

In [132]:
output.shape

torch.Size([2, 3, 286])

In [88]:
hidden[0].shape

torch.Size([2, 3, 128])

In [89]:
hidden[1].shape

torch.Size([2, 3, 128])

In [90]:
s_weights = output.squeeze().data.div(1.).exp()
s_weights.shape

torch.Size([3, 286])

In [93]:
s_idx = torch.multinomial(s_weights, 1)
s_idx

tensor([[ 7],
        [ 4],
        [48]])

In [108]:
s_weights_test = torch.tensor([0.2, 0.3, 0.5])
torch.multinomial(s_weights_test, 2)

tensor([0, 2])

In [124]:
s_weights_test = torch.tensor([2., 3., 5.])
torch.multinomial(s_weights_test, 10, replacement=True)

tensor([1, 0, 2, 2, 2, 1, 2, 1, 2, 0])

In [115]:
s_weights_test = torch.tensor([0.2, 0.3, 0.5])
s_weights_test

tensor([0.2000, 0.3000, 0.5000])